In [1]:
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [2]:
SEQUENCE_LIBRARY_DIR = '/data/experiments/YHE/sequence-library'
PERCOLATOR_FEATURES_FILE_NAME = "{}/percolator-id-feature-mapping.pkl".format(SEQUENCE_LIBRARY_DIR)


In [3]:
df = pd.read_pickle(PERCOLATOR_FEATURES_FILE_NAME)

In [4]:
df.columns

Index(['file_idx', 'scan', 'charge_x', 'spectrum precursor m/z',
       'spectrum neutral mass', 'peptide mass', 'percolator score',
       'percolator q-value', 'percolator PEP', 'total matches/spectrum',
       'sequence', 'protein id', 'flanking aa', 'feature_id', 'charge_y',
       'rt_apex', 'rt_peak_width', 'scan_apex', 'scan_peak_width', 'intensity',
       'precursor_id', 'monoisotopic_mass', 'predicted_mass_error',
       'recalibrated_monoisotopic_mass', 'recalibrated_monoisotopic_mz',
       'percolator_idx', 'human'],
      dtype='object')

In [9]:
df['protein id'].sample(n=50)

19576                                  sp|P26641|EF1G_HUMAN
15523                                 sp|Q9NQT4|EXOS5_HUMAN
370538                                 sp|P17987|TCPA_HUMAN
415688            sp|P00359|G3P3_YEAST,sp|P00358|G3P2_YEAST
443612                                 sp|P38606|VATA_HUMAN
441654                                 sp|P26640|SYVC_HUMAN
187359    sp|P57053|H2BFS_HUMAN,sp|Q99880|H2B1L_HUMAN,sp...
305154                                 sp|O43172|PRP4_HUMAN
87364                                  sp|P25789|PSA4_HUMAN
205398                                 sp|P53621|COPA_HUMAN
545315                                sp|P61981|1433G_HUMAN
378021                                 sp|Q92785|REQU_HUMAN
418022                                sp|P16949|STMN1_HUMAN
524481                                 sp|O14511|NRG2_HUMAN
94032                                  sp|P49915|GUAA_HUMAN
318251    sp|P57053|H2BFS_HUMAN,sp|Q99880|H2B1L_HUMAN,sp...
283110                                 s